# Interactive Brokers Connection Example

This notebook demonstrates basic connection and operations with Interactive Brokers using the ib_async library.

## Connection

**Important Note**: From now on, use `ib_async` instead of the older Interactive Brokers API.

In [3]:
from ib_async import *
util.startLoop()  # required in Jupyter (interactive environments), not in scripts

In [4]:
ib = IB()

**Make sure you are logged in your TWS!!**

In [5]:
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [6]:
ib.positions()

[]

In [7]:
ib.disconnect()

'Disconnecting from 127.0.0.1:7497, 117 B sent in 8 messages, 18.9 kB received in 398 messages, session time 10.5 s.'

## Contracts (Introduction)

Working with contracts is fundamental in Interactive Brokers. A contract represents a financial instrument and contains all the information needed to uniquely identify it on the exchange.

Here we'll create and qualify a Forex contract. The `qualifyContracts()` function fills in missing contract details by querying Interactive Brokers' database.

In [11]:
from ib_async import *
util.startLoop()

In [12]:
ib = IB()

In [13]:
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [14]:
contract = Forex("EURUSD")
contract

Forex('EURUSD', exchange='IDEALPRO')

Create a Forex contract for EUR/USD. The `Forex()` constructor creates a contract object, but it's not yet validated with IB:

In [15]:
contract = ib.qualifyContracts(contract)
contract

[Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')]

The `qualifyContracts()` function validates the contract with IB and fills in missing details like contract ID, exchange info, etc.:

In [16]:
type(contract)

list

Check the type - `qualifyContracts()` returns a list, even for a single contract:

In [17]:
contract[0]

Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')

Access the first (and only) contract in the returned list to see the full contract details:

In [18]:
contract[0].conId

12087792

The `conId` (contract ID) is a unique identifier assigned by Interactive Brokers to each financial instrument:

## Contract Types

Interactive Brokers supports various contract types. Each type has specific parameters:

- **Contract(conId)**: Direct reference using IB's contract ID
- **Stock**: Equity securities (symbol, exchange, currency, primaryExchange optional)
- **Forex**: Currency pairs (currency pair like 'EURUSD')
- **CFD**: Contracts for Difference (symbol)
- **Future**: Futures contracts (symbol, expiry date, exchange)
- **Option**: Options contracts (symbol, expiry, strike, right, exchange)
- **Bond**: Fixed income securities (using ISIN or other identifiers)

In [ ]:
# Contract(conId=270639)
# Stock('AMD', 'SMART', 'USD')
# Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')
# Forex('EURUSD')
# CFD('IBUS30')
# Future('ES', '20180921', 'GLOBEX')
# Option('SPY', '20170721', 240, 'C', 'SMART')
# Bond(secIdType='ISIN', secId='US03076KAA60')

### Handling Ambiguous vs Unambiguous Contracts

When creating contracts, some may be **ambiguous** (multiple matches) while others are **unambiguous** (single match):

- **Unambiguous**: When you provide enough details (symbol + exchange + currency), IB can uniquely identify the contract
- **Ambiguous**: When details are incomplete (e.g., just symbol), IB may find multiple matching contracts

The `qualifyContracts()` function behaves differently in each case:

In [19]:
contract = Stock("AAPL", "SMART", "USD")
contract

Stock(symbol='AAPL', exchange='SMART', currency='USD')

**Unambiguous case**: Full contract details provided (symbol, exchange, currency):

In [ ]:
contract = ib.qualifyContracts(contract)  # unambiguous
contract

This should return a single qualified contract since the details uniquely identify AAPL:

In [ ]:
contract = ib.qualifyContracts(Stock("AAPL"))  # ambiguous
contract

**Ambiguous case**: Only symbol provided, no exchange or currency specified. This may return multiple contracts if AAPL trades on different exchanges or in different currencies:

In [1]:
ib.disconnect()

NameError: name 'ib' is not defined

Finally, disconnect from Interactive Brokers when done: